In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm
import pickle

In [28]:
# Define se rodará em GPU ou CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# Carregar o conjunto de dados MNIST
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

# Definir hiperparâmetros
input_size = 784
hidden_sizes = [25, 50, 100]
learning_rates = [0.5, 1, 10]
algorithms = ["gradient_descent", "stochastic", "mini_batch"]
output_size = 10
num_epochs = 20

# Converter os dados para DataLoader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [11]:
# Definir a classe da rede neural
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [38]:
def experiment(hidden_size, learning_rate, algorithm):
    print("Hidden Size:", hidden_size)
    print("Learning Rate:", learning_rate)
    print("Algorithm:", algorithm)
    print()
    train_losses = []
    accuracies = []
    
    # Criar a rede neural
    model = NeuralNet(input_size, hidden_size, output_size)
    model.to(device)

    # Definir o otimizador e a função de perda
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Treinar a rede neural
    for epoch in range(num_epochs):
        print("Training!")
        print(f"Epoch:{epoch}")
        for i, (images, labels) in tqdm(enumerate(train_loader), total=len(train_dataset)):
            # Passing to Device
            labels = labels.to(device)
            images = images.to(device)
            
            # Reshape das imagens para um vetor unidimensional
            images = images.reshape(-1, input_size)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            if i % 10 == 0 or algorithm == "stochastic":
                # Backward pass e otimização
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        # Salva loss ao final de cada época 
        train_losses.append(loss.item())
        
        # Gradiente no final para o Gradiente Descendente
        if algorithm == "gradient_descent":
            # Backward pass e otimização
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # Testar a rede neural
        print("Evaluating!")
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in tqdm(test_loader):
                # Passing to Device
                labels = labels.to(device)
                images = images.to(device)

                images = images.reshape(-1, input_size)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total
        accuracies.append(accuracy)
    
    print("Done!\n---------------------------------------------------------")
    return [train_losses, accuracies]

In [39]:
experiments = []
for hidden_size in hidden_sizes:
    for algorithm in algorithms:
        for learning_rate in learning_rates:
            
            # Defining Experiment
            exp = {"hidden_size": hidden_size,
                   "algorithm": algorithm,
                   "learning_rate": learning_rate,
                   "results": experiment(hidden_size, learning_rate, algorithm)}
            
            # Adding to Answer
            experiments.append(exp)
            
            # Backup Save
            with open('results.pickle', 'wb') as handle:
                pickle.dump(experiments, handle, protocol=pickle.HIGHEST_PROTOCOL)

Hidden Size: 25
Learning Rate: 0.5
Algorithm: gradient_descent

Training!
Epoch:0


100%|███████████████████████████████████| 60000/60000 [00:17<00:00, 3436.22it/s]


Evaluating!


100%|███████████████████████████████████| 10000/10000 [00:02<00:00, 3544.53it/s]


Training!
Epoch:1


...

Training!
Epoch:19


100%|███████████████████████████████████| 60000/60000 [00:19<00:00, 3092.62it/s]


Evaluating!


100%|███████████████████████████████████| 10000/10000 [00:02<00:00, 3357.45it/s]

Done!
---------------------------------------------------------


In [40]:
learning_rates = [50, 20, 100]

experiments = []
for hidden_size in hidden_sizes:
    for algorithm in algorithms:
        for learning_rate in learning_rates:
            
            # Defining Experiment
            exp = {"hidden_size": hidden_size,
                   "algorithm": algorithm,
                   "learning_rate": learning_rate,
                   "results": experiment(hidden_size, learning_rate, algorithm)}
            
            # Adding to Answer
            experiments.append(exp)
            
            # Backup Save
            with open('results2.pickle', 'wb') as handle:
                pickle.dump(experiments, handle, protocol=pickle.HIGHEST_PROTOCOL)

Hidden Size: 25
Learning Rate: 50
Algorithm: gradient_descent

Training!
Epoch:0


100%|███████████████████████████████████| 60000/60000 [00:19<00:00, 3005.44it/s]


Evaluating!


100%|███████████████████████████████████| 10000/10000 [00:02<00:00, 3343.24it/s]


Training!
Epoch:1


...

Training!
Epoch:19


100%|███████████████████████████████████| 60000/60000 [00:19<00:00, 3117.50it/s]


Evaluating!


100%|███████████████████████████████████| 10000/10000 [00:02<00:00, 3442.22it/s]

Done!
---------------------------------------------------------
